In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

import os
import numpy as np
from pandas import DataFrame
import pandas as pd
import re
from notebook_utils import NotebookUtilities
from datetime import timedelta
import matplotlib.pyplot as plt
import re

nu = NotebookUtilities(data_folder_path=os.path.abspath('../data'))


# Build and deidentify the OSU dataset of Simulation Sessions

In [3]:

# Get all logs into one data frame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
else:
    
    # Add the CSVs to the data frame
    frvrs_logs_df = fu.concatonate_logs()
    
    # Remove numerically-named columns
    columns_list = [x for x in frvrs_logs_df.columns if not re.search(r'\d+', str(x))]
    frvrs_logs_df = frvrs_logs_df[columns_list]
    
    # Convert 'TRUE' and 'FALSE' to boolean values
    for cn in [
        'injury_record_injury_treated_with_wrong_treatment', 'injury_record_injury_treated',
        'injury_treated_injury_treated_with_wrong_treatment', 'injury_treated_injury_treated'
    ]:
        frvrs_logs_df[cn] = frvrs_logs_df[cn].map({'TRUE': True, 'FALSE': False, 'True': True, 'False': False})
    
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
print(frvrs_logs_df.shape) # (842663, 106)
columns_list = [cn for cn in frvrs_logs_df.columns if 'appl' in cn]
mask_series = False
for cn in columns_list: mask_series |= ~frvrs_logs_df[cn].isnull()
df = frvrs_logs_df[mask_series][columns_list]
display(df.sample(min(4, df.shape[0])).dropna(axis='columns', how='all').T)

(829277, 114)


,770431,135877,129450,652629
tag_applied_patient_id,Bob_0 Root,Helga_0 Root,Mike_6 Root,Gary_3 Root
tag_applied_type,black,red,red,gray



## Check for duplicate file ingestion

In [5]:

# Filter all the rows that have more than one unique value in the file_name column for each value in the session_uuid column
mask_series = (frvrs_logs_df.groupby('session_uuid').file_name.transform(pd.Series.nunique) > 1)
assert frvrs_logs_df[mask_series].shape[0] == 0, "You have duplicate files"


## Add new features according to your increasing domain knowledge

In [6]:

# Modalize into one patient ID column if possible
new_column_name = 'patient_id'
if (new_column_name not in frvrs_logs_df.columns):
    columns_list= [
        'patient_demoted_id', 'patient_record_id', 'injury_record_patient_id', 's_a_l_t_walk_if_can_patient_id',
        's_a_l_t_walked_patient_id', 's_a_l_t_wave_if_can_patient_id', 's_a_l_t_waved_patient_id', 'patient_engaged_id',
        'pulse_taken_patient_id', 'injury_treated_patient_id', 'tool_applied_patient_id', 'tag_applied_patient_id',
        'player_gaze_patient_id'
    ]
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, columns_list, new_column_name)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (842663, 107)

Pickling to C:\Users\DaveBabbitt\Documents\GitHub\itm-analysis-reporting\saves\pkl\frvrs_logs_df.pkl
(829277, 107)


In [7]:

# Modalize into one location ID column if possible
new_column_name = 'location_id'
if (new_column_name not in frvrs_logs_df.columns):
    columns_list= [
        'teleport_location', 'patient_demoted_position', 'patient_record_position', 'injury_record_injury_injury_locator',
        's_a_l_t_walk_if_can_sort_location', 's_a_l_t_walked_sort_location', 's_a_l_t_wave_if_can_sort_location',
        's_a_l_t_waved_sort_location', 'patient_engaged_position', 'bag_access_location', 'injury_treated_injury_injury_locator',
        'bag_closed_location', 'tag_discarded_location', 'tool_discarded_location', 'player_location_location',
        'player_gaze_location'
    ]
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, columns_list, new_column_name)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (842663, 108)

Pickling to C:\Users\DaveBabbitt\Documents\GitHub\itm-analysis-reporting\saves\pkl\frvrs_logs_df.pkl
(829277, 108)


In [8]:

# Modalize into one injury ID column if possible
new_column_name = 'injury_id'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, ['injury_record_id', 'injury_treated_id'], new_column_name)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (842663, 109)

Pickling to C:\Users\DaveBabbitt\Documents\GitHub\itm-analysis-reporting\saves\pkl\frvrs_logs_df.pkl
(829277, 109)


In [30]:

# Modalize into one patient sort column if possible
new_column_name = 'patient_sort'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, ['patient_demoted_sort', 'patient_record_sort', 'patient_engaged_sort'], new_column_name)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829277, 109)

Pickling to C:\Users\DaveBabbitt\Documents\GitHub\itm-analysis-reporting\saves\pkl\frvrs_logs_df.pkl
(829277, 114)


In [9]:

# Add a voice capture sentiment score
if ('voice_capture_sentiment_score' not in frvrs_logs_df.columns):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    sid = SentimentIntensityAnalyzer()
    mask_series = frvrs_logs_df.voice_capture_message.isnull()
    for row_index, row_series in frvrs_logs_df[~mask_series].iterrows():
        voice_capture_message = '\n' + row_series.voice_capture_message
        frvrs_logs_df.loc[row_index, 'voice_capture_sentiment_score'] = sid.polarity_scores(voice_capture_message)['compound']
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (842663, 110)

Pickling to C:\Users\DaveBabbitt\Documents\GitHub\itm-analysis-reporting\saves\pkl\frvrs_logs_df.pkl
(829277, 110)


In [10]:

# Mask voice capture PII
# OSU screened all of the **VOICE_COMMAND** and **VOICE_CAPTURE** lines and
# replaced any names with either Max or Jane, regardless of whether the name was that of the responder.
# But, just to make sure...
columns_list = ['voice_command_command_description', 'voice_capture_message']
if not frvrs_logs_df[columns_list].applymap(lambda x: '[PERSON]' in str(x), na_action='ignore').sum().sum():
    import spacy
    nlp = spacy.load('en_core_web_sm')
    import en_core_web_sm
    nlp = en_core_web_sm.load()
    
    mask_series = frvrs_logs_df.voice_command_command_description.isnull() & frvrs_logs_df.voice_capture_message.isnull()
    df = frvrs_logs_df[~mask_series]
    def mask_pii(srs):
        for idx in columns_list:
            new_text = srs[idx]
            if str(new_text) != 'nan':
                doc = nlp(new_text)
                for entity in doc.ents:
                    if entity.label_ == 'PERSON':
                        new_text = re.sub('\\b' + entity.text + '\\b', '[PERSON]', new_text)
                srs[idx] = new_text
    
        return srs
    
    for row_index, row_series in df.apply(mask_pii, axis='columns')[columns_list].iterrows():
        for column_name, column_value in row_series.items():
            if str(column_value) != 'nan':
                frvrs_logs_df.loc[row_index, column_name] = column_value
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (842663, 110)

Pickling to C:\Users\DaveBabbitt\Documents\GitHub\itm-analysis-reporting\saves\pkl\frvrs_logs_df.pkl
(829277, 110)


In [11]:

# Any runs longer than that 16 minutes are probably an instance
# of someone taking off the headset and setting it on the ground.
# 1 second = 1,000 milliseconds; 1 minute = 60 seconds
new_column_name = 'is_scene_aborted'
if (new_column_name in frvrs_logs_df.columns): frvrs_logs_df = frvrs_logs_df.drop(columns=new_column_name)
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df[new_column_name] = False
    for (session_uuid, scene_index), scene_df in frvrs_logs_df.groupby(fu.scene_groupby_columns):
        mask_series = True
        for cn in fu.scene_groupby_columns: mask_series &= (frvrs_logs_df[cn] == eval(cn))
        frvrs_logs_df.loc[mask_series, new_column_name] = fu.get_is_scene_aborted(scene_df)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (842663, 111)
    display(frvrs_logs_df.groupby('is_scene_aborted').size().to_frame().rename(columns={0: 'count'}))

Pickling to C:\Users\DaveBabbitt\Documents\GitHub\itm-analysis-reporting\saves\pkl\frvrs_logs_df.pkl
(829277, 111)


,count
is_scene_aborted,
False,829277


In [12]:

# Check if all the patient IDs in any run are some variant of Mike and designate those runs as "Orientation"
if ('scene_type' not in frvrs_logs_df.columns): frvrs_logs_df['scene_type'] = 'Triage'
column_value = 'Orientation'
if (column_value not in frvrs_logs_df.scene_type.unique()):
    
    # Filter out those files from the dataset and mark them
    base_mask_series = frvrs_logs_df.groupby(fu.scene_groupby_columns).patient_id.transform(lambda srs: all(srs.str.lower().str.contains('mike')))
    frvrs_logs_df.loc[base_mask_series, 'scene_type'] = column_value
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (842663, 113)

Pickling to C:\Users\DaveBabbitt\Documents\GitHub\itm-analysis-reporting\saves\pkl\frvrs_logs_df.pkl
(829277, 112)


In [13]:

# Get a sample with a clear count of responders
new_column_name = 'is_a_one_triage_file'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df[new_column_name] = False
    for file_name in frvrs_logs_df.file_name.unique():
        is_a_one_triage_file = fu.get_is_a_one_triage_file(frvrs_logs_df, file_name)
        mask_series = (frvrs_logs_df.file_name == file_name)
        frvrs_logs_df.loc[mask_series, new_column_name] = is_a_one_triage_file
    
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (842663, 111)

Pickling to C:\Users\DaveBabbitt\Documents\GitHub\itm-analysis-reporting\saves\pkl\frvrs_logs_df.pkl
(829277, 113)



## Remove training sessions

In [14]:

import docx

logs_folder = nu.data_logs_folder
for sub_directory, directories_list, files_list in os.walk(logs_folder):
    for file_name in files_list:
        if file_name.endswith('.docx'):
            file_path = os.path.join(sub_directory, file_name)
            print(file_path)
            doc = docx.Document(file_path)
            text = ''
            for paragraph in doc.paragraphs: text += '\n' + paragraph.text
            print(text)

C:\Users\DaveBabbitt\Documents\GitHub\itm-analysis-reporting\data\logs\All CSV files renamed by date\Notes.docx

DCEMS 11.22 – 12.22 – OK
Madison Township 4.18.23 and 4.20.23 – Training and Triage are combined.
Union County Marysville 04.25.23 - Training and Triage are combined.
Disaster Day 3.14.23 and 3.15.23 are almost all combined.




## We need to add a _responder_type_:

EMT = EMT Basic

AEMT = EMT Advanced or Intermediate

MEDIC = Paramedic

STDNT = Any Medical Student

RSDNT = Any Resident

FELO = Any EM Fellow

ATEND = Any Attending

In [15]:

# Define the custom categorical orders
responder_types = ['EMT', 'AEMT', 'MEDIC', 'STDNT', 'RSDNT', 'FELO', 'ATEND']
responder_types_category_order = pd.CategoricalDtype(categories=responder_types, ordered=True)